In [1]:
import urllib.request
import urllib.error
import aiohttp
import asyncio


def urllib_download(download_url, filetype):
    """
    Use the urllib Request and Error packages to download the contents of
        a webpage, if it exists
    
    Arguments:
        download_url (str): Link to the webpage to download the desired file.
        filetype (str): Either "ENDF" or "PENDF" filetypes to be downloaded
            (case insensitive).
    """

    try:
        with urllib.request.urlopen(download_url) as f:
            temp_file = f.read().decode('utf-8')
    except urllib.error.URLError as e:
        if e.code == 404:
            temp_file = None
            raise FileNotFoundError(f'{filetype.upper()} file does not exist at {download_url}')
    return temp_file

async def fetch(session, url):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.text()
            else:
                return None
    except aiohttp.ClientError:
        return None

async def fetch_with_sem(semaphore, session, url):
    async with semaphore:
        return await fetch(session, url)

async def async_tendl_download(element, concurrency_limit = 50, iterate=True):
    downloads = []
    A_vals = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    async with aiohttp.ClientSession() as session:
        tasks = []
        for A in range(1, 292):  # Adjusted to range from 1 to 291 (inclusive)
            for i in range(2):
                A_str = str(A).zfill(3)
                if i == 1:
                    A_str += 'm'

                isotope_component = f'{element}/{element}{A_str}/lib/endf/n-{element}{A_str}.'
                tendl_gen_url = 'https://tendl.web.psi.ch/tendl_2017/neutron_file/'
                tendl_url = tendl_gen_url + isotope_component + 'tendl'
                pendf_url = tendl_gen_url + isotope_component + 'pendf'

                tendl_task = fetch_with_sem(semaphore, session, tendl_url)
                pendf_task = fetch_with_sem(semaphore, session, pendf_url)

                tasks.append((tendl_task, pendf_task, tendl_url, pendf_url, A_str))

        results = await asyncio.gather(*[asyncio.gather(tendl_task, pendf_task) for tendl_task, pendf_task, _, _, _ in tasks])

        for (tendl_result, pendf_result), (tendl_task, pendf_task, tendl_url, pendf_url, A_str) in zip(results, tasks):
            if tendl_result and pendf_result:
                downloads.append((tendl_url, pendf_url))
                A_vals.append(A_str)
            elif not iterate:
                logger.error(FileNotFoundError())

    return A_vals

def download_tendl(element, A, filetype, save_path = None):
    """
    Download ENDF/PENDF files from the TENDL 2017 database for specific isotopes.

    Arguments:
        element (str): Chemical symbol for element of interest (i.e. Ti).
        A (str or int): Mass number for selected isotope (i.e. 48).
            If the target is an isomer, "m" after the mass number (i.e. 48m),
            so A must be input as a string.
        filetype (str): Either "ENDF" or "PENDF" filetypes to be downloaded
            (case insensitive).
        save_path (str, optional): User-defined file path for the downloaded file.
            Defaults to None and will be otherwise defined internally.
    
    Returns:
        save_path (str): File path for the file downloaded file.
    """

    # Ensure that A is properly formatted
    A = str(A).zfill(3)
#    if 'm' in A:
#        A += 'm'

    # Create a dictionary to generalize formatting for both ENDF and PENDF files
    file_handling = {'endf' : {'ext': 'tendl', 'tape_num': 20},
                     'pendf' : {'ext': 'pendf', 'tape_num': 21}}
    
    # Construct the filetype and isotope specific URL
    isotope_component = f'{element}/{element}{A}/lib/endf/n-{element}{A}.'
    ext = file_handling[filetype.lower()]['ext']
    tendl_gen_url = 'https://tendl.web.psi.ch/tendl_2017/neutron_file/'
    download_url = tendl_gen_url + isotope_component + ext
    print(f'{filetype.upper()} URL: {download_url}')

    # Define a save path for the file if there is not one already specified
    if save_path is None:
        save_path = f'tape{file_handling[filetype.lower()]["tape_num"]}'

    # Conditionally downloa
    temp_file = urllib_download(download_url, filetype)

    with open(save_path, 'w') as f:
        f.write(temp_file)

    return save_path

In [4]:
pendf_path = download_tendl('C', 13, 'pendf')
endf_path = download_tendl('C', 13, 'endf')

PENDF URL: https://tendl.web.psi.ch/tendl_2017/neutron_file/C/C013/lib/endf/n-C013.pendf
ENDF URL: https://tendl.web.psi.ch/tendl_2017/neutron_file/C/C013/lib/endf/n-C013.tendl


In [6]:
import ENDFtk
def extract_endf_specs(path, filetype):
    """
    Extract the material ID and MT numbers from an ENDF or GENDF file.

    Arguments:
        path (str): File path to the selected ENDF/GENDF file.
        filetype (str): Either ENDF or GENDF (case insensitive)
    
    Returns:
        matb (int): Unique material ID extracted from the file.
        MTs (list): List of reaction types (MT's) present in the file.
        file (ENDFtk.tree.File or None): ENDFtk file object containing the contents
            for a specific material's cross-section data.
            Only returns the file for GENDF filetypes.
    """
    # Read in ENDF tape using ENDFtk
    tape = ENDFtk.tree.Tape.from_file(path)

    # Determine the material ID
    mat_ids = tape.material_numbers
    matb = mat_ids[0]

    # Set MF for cross sections
    xs_MF = 3

    # Extract out the file
    file = tape.material(matb).file(xs_MF)

    # Extract the MT numbers that are present in the file
    MTs = [MT.MT for MT in file.sections.to_list()]

    filetype = filetype.lower()
    return_values = {
        'endf': (matb, MTs),
        'gendf': (matb, MTs, file)
    }
    return return_values.get(filetype)


In [8]:
material_id, MTs = extract_endf_specs(endf_path, 'endf')

In [9]:
# Import packages
import subprocess

# Define constants
NENDF = 20 # unit for endf tape
NPEND = 21 # unit for pendf tape
NGOUT1 = 0 # unit for input gout tape (default=0)
NGOUT2 = 31 # unit for output gout tape (default=0)
IGN = 17 # neutron group structure option (corresponding to Vitamin J)
IGG = 0 # gamma group structure option
IWT = 11 # weight function option (corresponding to Vitamin E)
LORD = 0 # Legendre order
NTEMP = 1 # number of temperatures (default=1)
NSIGZ = 1 # number of sigma zeroes (default=1)
IPRINT = 1 # long print option (0/1=minimum/maximum) --(default=1)
ISMOOTH = 1 # switch on/off smoother operation (1/0, default=1=on)
TEMP = 293.16 # temperature in Kelvin
SIGZ = 0 # sigma zero values (including infinity)
MFD = 3 # file to be processed
MATD = 0 # next mat number to be processed

# Dictionary of elements in the Periodic Table
elements = [
    'H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
    'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca',
    'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr',
    'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
    'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
    'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
    'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th',
    'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm',
    'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds',
    'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og'
]
elements = dict(zip(elements, range(1, len(elements)+1)))

def format_card(card_number, card_content, MTs):
    """
    Format individual "cards" for the NJOY input cards to run with GROUPR.
        Formatting and terminology based on the NJOY user manual:
        (https://github.com/njoy/NJOY2016-manual/raw/master/njoy16.pdf)

    Arguments:
        card_number (int): Individual identifier of "card" in input "deck".
        card_content (list): Values to be written on each individual "card".
        MTs (list): List of reaction types (MT's) present in the ENDF/PENDF files.
    
    Returns:
        card_str (str): Concatenated string of an individual "card's" contents. 
    """
    
    # Initialize string and concatenate contents of the card with it
    card_str = ''
    gen_str = ' ' + ' '.join(map(str, card_content))
    if card_number == 9:
        card_str = ' ' + '/\n '.join(card_content) + '/\n'
    elif card_number == 4:
        card_str += gen_str + '\n'
    else:
        card_str += gen_str + '/\n'
    return card_str

def groupr_input_file_format(matb, MTs, element, A, mt_dict):

    """"
    Format the input "deck" to run NJOY from individually formatted cards.
        Most parameters are predefined as constants, as necessary for a GROUPR run
        to write a group-wise file (GENDF) based on the ENDF and PENDF files
        with a Vitamin-J 175 group structure and a Vitamin-E weighting function.
        Other parameters are isotope specific, such as "matb", which corresponds
        to the NJOY naming convention for the material ID, which are separately
        extracted from the ENDF file to be referenced.

        Formatting and terminology based on the NJOY user manual:
        (https://github.com/njoy/NJOY2016-manual/raw/master/njoy16.pdf)

    Arguments:
        matb (int): Unique material ID extracted from the ENDF base file.
        MTs (list): List of reaction types (MT's) present in the ENDF/PENDF files.
        element (str): Chemical symbol for element of interest.
        A (str or int): Mass number for selected isotope.
            If the target is an isomer, "m" after the mass number,
            so A must be input as a string.
        mt_dict (dict): Reference dictionary containing reaction information
            for each MT number pre-defined in the ENDF manual.
            (https://www.oecd-nea.org/dbdata/data/manual-endf/endf102_MT.pdf)
    
    Returns:
        cards (dict): Dictionary containing each "card" identified by its card number.
    """

    cards = {}
    
    cards[1] = [NENDF, NPEND, NGOUT1, NGOUT2]
    # matb -- (already defined) -- material to be processed
    cards[2] = [matb, IGN, IGG, IWT, LORD, NTEMP, NSIGZ, IPRINT]
    Z = str(elements[element]).zfill(2)
    title = f'"{Z}-{element}-{A} for TENDL 2017"'
    cards[3] = [title]
    cards[4] = [TEMP]
    cards[5] = [SIGZ]
    mtd = MTs # sections to be processed
    cards[9] = []
    for MT in MTs:
        mtname = mt_dict[str(MT)] # description of section to be processed
        card9_line = f'{MFD} {MT} "{mtname}"'
        cards[9].append(card9_line)
    cards[10] = [MATD]

    return cards

def groupr_input_file_writer(cards, MTs):
    """"
    Write out the NJOY GROUPR input card by formatting each card line by line.

    Arguments:
        cards (dict): Dictionary containing each "card" identified by its card number.
        MTs (list): List of reaction types (MT's) present in the ENDF/PENDF files.
    
    Returns:
        None
    """

    # Write the input deck to the groupr.inp file
    with open('groupr.inp', 'w') as f:
        f.write('groupr\n')
        for card_num, card in cards.items():
            f.write(format_card(card_num, card, MTs))
        f.write(' 0/\nstop')

def run_njoy(cards, element, A):
    """
    Use subprocess to run NJOY given a pre-written input card to convert a pair
        of ENDF and PENDF files to a GENDF file and save it locally.
    
    Arguments:
        cards (dict): Dictionary containing each "card" identified by its card number.
        element (str): Chemical symbol for element of interest.
        A (str or int): Mass number for selected isotope.
            If the target is an isomer, "m" after the mass number,
            so A must be input as a string.
    
    Returns:
        gendf_path (str): File path to the newly created GENDF file. 
    """

    # Define the input files
    INPUT = 'groupr.inp'
    OUTPUT = 'groupr.out'

    # Run NJOY
    result = subprocess.run(['njoy'], input=open(INPUT).read(), text=  True, capture_output=True)
    with open(OUTPUT, 'w') as output_file:
        output_file.write(result.stdout)

    # If the run is successful, log out the output and make a copy of the file as a .GENDF file
    if result.stderr == '':
        output = subprocess.run(['cat', 'output'], capture_output=True, text = True)
        title = cards[3][0][1:-1]
        title_index = output.stdout.find(title)
        #print(f'\n{output.stdout[:title_index + len(title)]}\n')

        gendf_path = f'./gendf_files/tendl_2017_{element}{A}.gendf'
        subprocess.run(['cp', 'tape31', gendf_path])
        return gendf_path
    else:
        print(result.stderr)

def njoy_file_cleanup(output_path = 'njoy_ouput'):
    """
    Clean up repository from unnecessary intermediate files from NJOY run.

    Arguments:
        output_path (str): The path where the automated NJOY output will be saved.
            Defaults to 'njoy_output'.
    
    Returns:
        None
    """

    njoy_files = ['groupr.inp', 'groupr.out', 'tape20', 'tape21', 'tape31']
    for file in njoy_files:
        if file == 'groupr.out':
            with open(file, 'r') as f:
                groupr_out = f.read()
            print(groupr_out)
            
        subprocess.run(['rm', file])
    subprocess.run(['mv', 'output', output_path])
    #(f'Full NJOY output file readout can be found at {output_path}')

In [11]:
import csv
def read_csv(csv_path):
    """
    Read in the mt_table.csv file and store it in a dictionary.
    
    Arguments:
        csv_path (str): File path to mt_table.csv
            This should be in the same repository.
    
    Returns:
        data_dict (dict): Dictionary formatted data structure for mt_table.csv
    """
    mt_dict = {}

    with open(csv_path, 'r') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:
            mt_dict[row['MT']] = row['Reaction']

    return mt_dict

mt_dict = read_csv('mt_table.csv')

In [13]:
card_deck = groupr_input_file_format(material_id, MTs, 'C', 13, mt_dict)
groupr_input_file_writer(card_deck, MTs)

In [14]:
run_njoy(card_deck, 'C', 13)

STOP 77



In [10]:
import numpy as np
#           emitted             delta N       delta P
delta_NP = {'n'     : np.array([-1      ,      0     ]),
            'p'     : np.array([ 0      ,     -1     ]),
            't'     : np.array([-2      ,     -1     ]),
            '3He'   : np.array([-1      ,     -2     ]),
            'α'     : np.array([-2      ,     -2     ]),
}
n,p = delta_NP['n']

In [13]:
delta_NP.items()

dict_items([('n', array([-1,  0])), ('p', array([ 0, -1])), ('t', array([-2, -1])), ('3He', array([-1, -2])), ('α', array([-2, -2]))])

In [103]:
from activation_analysis import count_emitted_particles, isomer_check

def emission_breakdown(emitted_particles):
    particle_types = ['n', 'd', 'α', 'p', 't', '3He', 'gamma']
    emission_dict = {
        particle : count_emitted_particles(particle, emitted_particles)
        for particle in particle_types
        if particle in emitted_particles
    }
    return emission_dict

In [104]:
emission_breakdown('2n24d3He')

{'n': 2, 'd': 24, '3He': 1}

In [2]:
def nucleon_changes(emission_dict):
    NP_change = np.array([1, 0]) # neutron activation
    
    #           emitted             delta N       delta P
    NP_dict = {'n'     : np.array([-1      ,      0     ]), # neutron emission
                'p'     : np.array([ 0      ,     -1     ]), # proton emission
                'd'     : np.array([-1      ,     -1     ]), # deuteron emission
                't'     : np.array([-2      ,     -1     ]), # triton emission
                '3He'   : np.array([-1      ,     -2     ]), # helium-3 emission
                'α'     : np.array([-2      ,     -2     ]), # alpha particle emission
                }

    for particle, count in emission_dict.items():
        NP_change += count * delta_NP[particle]
        
    return NP_change

In [1]:
import csv
def load_csv(csv_path):
    """
    Load in the mt_table.csv file and store it in a dictionary.
    
    Arguments:
        csv_path (str): File path to mt_table.csv
            This should be in the same repository.
    
    Returns:
        mt_dict (dict): Dictionary formatted data structure for mt_table.csv
    """
    mt_dict = {}

    with open(csv_path, 'r') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:
            emitted_particles = row['Reaction'].split(',')[1][:-1]
            emission_dict = emission_breakdown(emitted_particles)
            change_N, change_P = nucleon_changes(emission_dict)
            M = isomer_check(emitted_particles)
            mt_dict[row['MT']] = {
                'Reaction' : row['Reaction'],
                'delKZA' : (change_P * 1000 + change_P + change_N) * 10 + M,
                'Emitted Particles' : emitted_particles
            }

    return mt_dict

mt_dict = load_csv('mt_table.csv')

NameError: name 'emission_breakdown' is not defined

In [129]:
mt_dict

{'1': {'Reaction': '(n, total)',
  'delKZA': -10020,
  'Emitted Particles': ' total'},
 '2': {'Reaction': '(z,z0)', 'delKZA': 10, 'Emitted Particles': 'z0'},
 '3': {'Reaction': '(z, nonelas.)',
  'delKZA': 0,
  'Emitted Particles': ' nonelas.'},
 '4': {'Reaction': '(z,n)', 'delKZA': 0, 'Emitted Particles': 'n'},
 '5': {'Reaction': '(z,anything)',
  'delKZA': -10030,
  'Emitted Particles': 'anything'},
 '10': {'Reaction': '(z,contin.)',
  'delKZA': -10030,
  'Emitted Particles': 'contin.'},
 '11': {'Reaction': '(z,2nd)', 'delKZA': -10030, 'Emitted Particles': '2nd'},
 '16': {'Reaction': '(z,2n)', 'delKZA': -10, 'Emitted Particles': '2n'},
 '17': {'Reaction': '(z,3n)', 'delKZA': -20, 'Emitted Particles': '3n'},
 '18': {'Reaction': '(z,fission)',
  'delKZA': 0,
  'Emitted Particles': 'fission'},
 '19': {'Reaction': '(n,f)', 'delKZA': 10, 'Emitted Particles': 'f'},
 '20': {'Reaction': '(n,nf)', 'delKZA': 0, 'Emitted Particles': 'nf'},
 '21': {'Reaction': '(n,2nf)', 'delKZA': -10, 'Emitted 

In [133]:
mt_dict['11'].keys()

dict_keys(['Reaction', 'delKZA', 'Emitted Particles'])

In [127]:
MT = 11
mt_dict[str(MT)]['Reaction']

'(z,2nd)'

In [116]:
pKZA = 260560
dKZA = pKZA + delKZA
dKZA

230510

In [95]:
nuclear_decay({'n': 1, 'α': 1, 'p': 1})

array([-3, -3])

In [68]:
Z = 26
A = 56
n = A - Z
M = 0
KZA = (Z * 1000 + Z + n) * 10 + M
KZA

260560

In [17]:
A = '56M for tendl 2017                                              0 0  0    0'
A.lower().split(' ')[0].split('m')[0]

'56'

In [ ]:
MT3_FEND = '                                                                  2631 0  0    0'

In [3]:
spaces = '                                                                  '
len(spaces)

66

In [4]:
' ' * 66 == spaces

True

In [20]:
gendf_path = '/filespace/e/esweinstein2/ALARA/src/DataLib/fendl32B_retrofit/gendf_files/tendl_2017_Fe56.gendf'
whitespace = ' ' * 66
matd = str(2631)
with open(gendf_path, 'r') as gendf_file:
    file_str = gendf_file.read()
identifier = f'{matd} 1451   '
mf1_last_line = file_str.rfind(identifier)
file_str[mf1_last_line + len(identifier):mf1_last_line + len(identifier) + 2]
new_line = f'{whitespace}{matd} 1 0'
with open(gendf_path, 'w') as gendf_file:


'                                                                  2631 1 0'

In [23]:
gendf_path = '/filespace/e/esweinstein2/ALARA/src/DataLib/fendl32B_retrofit/gendf_files/tendl_2017_Fe56.gendf'
whitespace = ' ' * 66
matb = str(2631)

# Read the file content
with open(gendf_path, 'r') as gendf_file:
    file_str = gendf_file.read()

mf1_identifier = f'{matb} 1451   '
mf3_identifier = f'{matb} 3  099999'
mf1_last_line_index = file_str.rfind(mf1_identifier)
mf3_SEND_RECORD_index = file_str.rfind(mf3_identifier)

# Prepare the new line
mf1_SEND_RECORD = f'{whitespace}{matb} 1  099999'
mf3_FEND_RECORD = f'{whitespace}{matb} 0  0    0'

# Calculate the end position of the identified line
end_of_identifier_line = file_str.find('\n', mf1_last_line_index) + 1

# Split the file content into lines
file_lines = file_str.splitlines()

# Find the line number of the identified line
line_number = file_str[:mf1_last_line_index].count('\n')

# Insert the new line after the identified line
file_lines.insert(line_number + 1, mf1_SEND_RECORD)

# Join the lines back into a single string
new_file_str = '\n'.join(file_lines) + '\n'

# Write the updated content back to the file
with open(gendf_path, 'w') as gendf_file:
    gendf_file.write(new_file_str)

In [24]:
gendf_path = '/filespace/e/esweinstein2/ALARA/src/DataLib/fendl32B_retrofit/gendf_files/tendl_2017_Fe56.gendf'
matb = str(2631)
whitespace = ' ' * 66

# Define identifiers and corresponding new lines
mf1_identifier = f'{matb} 1451   '
mf3_identifier = f'{matb} 3  099999'
mf1_SEND_RECORD = f'{whitespace}{matb} 1  099999'
mf3_FEND_RECORD = f'{whitespace}{matb} 0  0    0'

# Store identifiers and new lines in lists
identifiers = [mf1_identifier, mf3_identifier]
new_lines = [mf1_SEND_RECORD, mf3_FEND_RECORD]

# Read the file content
with open(gendf_path, 'r') as gendf_file:
    file_str = gendf_file.read()

# Initialize a list to hold modified file lines
file_lines = file_str.splitlines()

# Iterate over each identifier and corresponding new line
for identifier, new_line in zip(identifiers, new_lines):
    last_line_index = file_str.rfind(identifier)
    
    # Calculate the end position of the identified line
    end_of_identifier_line = file_str.find('\n', last_line_index) + 1

    # Find the line number of the identified line
    line_number = file_str[:last_line_index].count('\n')

    # Insert the new line after the identified line
    file_lines.insert(line_number + 1, new_line)

# Join the lines back into a single string
new_file_str = '\n'.join(file_lines) + '\n'

# Write the updated content back to the file
with open(gendf_path, 'w') as gendf_file:
    gendf_file.write(new_file_str)

In [26]:
def ensure_gendf_markers(gendf_path, matb):
    """
    Edit the GENDF files produced from an NJOY GROUPR run to include file and
    section records that are not automatically written out to the file by
    NJOY. Missing these records will not cause errors, but will raise
    messages when they are read by ENDFtk, which expects these markers, so
    this method ensures that they are present. Edits will only be made if
    the SEND record for MF1 or the FEND record for MF3 are not present.
    The formatting for these records can be found at:
    https://t2.lanl.gov/nis/endf/intro06.html

    Arguments:
        gendf_path (str): File path to the newly created GENDF file.
        matb (int): Unique material ID for the material in the GENDF file.
    
    Returns:
        None
    """

    # In ENDF-6 formatted files, there are 66 lines of whitespace before
    # the values in record-keeping lines
    whitespace = ' ' * 66

    # Define identifiers with original variable names
    mf1_identifier = f'{matb} 1451   '
    mf3_identifier = f'{matb} 3  099999'

    # Define new lines with original variable names
    mf1_SEND_RECORD = f'{whitespace}{matb} 1  099999'
    mf3_FEND_RECORD = f'{whitespace}{matb} 0  0    0'

    updates = [
        (mf3_identifier, mf3_FEND_RECORD),
        (mf1_identifier, mf1_SEND_RECORD)
    ]

    with open(gendf_path, 'r') as gendf_file:
        file_str = gendf_file.read()

    file_lines = file_str.splitlines()

    for identifier, new_line in updates:
        if identifier not in file_str:
            last_line_index = file_str.rfind(identifier)
            end_of_identifier_line = file_str.find('\n', last_line_index) + 1
            line_number = file_str[:last_line_index].count('\n')
            file_lines.insert(line_number + 1, new_line)

    new_file_str = '\n'.join(file_lines) + '\n'

    with open(gendf_path, 'w') as gendf_file:
        gendf_file.write(new_file_str)
ensure_gendf_markers('/filespace/e/esweinstein2/ALARA/src/DataLib/fendl32B_retrofit/gendf_files/tendl_2017_Ti48m.gendf', 2232)

In [1]:
help(zip)

Help on class zip in module builtins:

class zip(object)
 |  zip(*iterables) --> A zip object yielding tuples until an input is exhausted.
 |  
 |     >>> list(zip('abcdefg', range(3), range(4)))
 |     [('a', 0, 0), ('b', 1, 1), ('c', 2, 2)]
 |  
 |  The zip object yields n-length tuples, where n is the number of iterables
 |  passed as positional arguments to zip().  The i-th element in every tuple
 |  comes from the i-th iterable argument to zip().  This continues until the
 |  shortest argument is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and ret

In [3]:
import requests
from bs4 import BeautifulSoup

def fetch_and_parse_html(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to fetch the webpage. Status code: {response.status_code}")
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Print the title of the webpage
    title = soup.title.string if soup.title else 'No title found'
    print(f"Title of the webpage: {title}")
    
    # Find all <a> tags and extract the href attributes
    links = soup.find_all('a')
    urls = [link.get('href') for link in links if link.get('href')]
    
    # Print all found URLs
    for i, url in enumerate(urls, 1):
        print(f"URL {i}: {url}")
    
    # Return the list of URLs
    return urls

# Example usage
url = 'https://tendl.web.psi.ch/tendl_2017/neutron_html/Be/NeutronBe07.html'
urls = fetch_and_parse_html(url)

Title of the webpage: Neutron sub-library - TENDL-2017
URL 1: NeutronBe07angle.html
URL 2: NeutronBe07angle1.html
URL 3: NeutronBe07gamma.html
URL 4: NeutronBe07residual.html
URL 5: NeutronBe07spectra.html
URL 6: NeutronBe07xs.html
URL 7: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/endf/n-Be007.tendl
URL 8: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/endf/n-Be007.pendf
URL 9: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/endf/n-Be007.ace.gz
URL 10: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/endf/n-Be007.xsdir
URL 11: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/endfs20/n-Be007-s20.tendl
URL 12: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/eaf/n-Be007-eaf.tendl
URL 13: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/eaf/n-Be007-eaf.tendl_cov
URL 14: https://tendl.web.psi.ch/tendl_2017/neutron_file/Be/Be007/lib/acf/n-Be007-acf.tendl
URL 15: https://tendl.web.psi.ch/tendl

In [6]:
TENDL_GEN_URL = 'https://tendl.web.psi.ch/tendl_2017'
element = 'Be'

for A in range(1,292):
    for i in range(2):
        A_str = str(A).zfill(3)
        if i == 1:
            A_str += 'm'
        
        navigation_page_url = f'{TENDL_GEN_URL}/neutron_html/{element}/Neutron{element}{str(A).zfill(2)}.html'
        isotope_component = f'/neutron_file/{element}/{element}{A_str}/lib/endf/n-{element}{A_str}.'
        tendl_url = TENDL_GEN_URL + isotope_component + 'tendl'
        pendf_url = TENDL_GEN_URL + isotope_component + 'pendf'
        
        nav_urls = fetch_and_parse_html(navigation_page_url)

        if tendl_url in nav_urls and pendf_url in nav_urls:
            print(A_str)

Exception: Failed to fetch the webpage. Status code: 404

In [24]:
import subprocess

element = 'Fe'
# Run the command
gendf_files = subprocess.run(f'ls gendf_files/*{element}*',
                        shell=True,
                        check=True,
                        stdout=subprocess.PIPE,
                        universal_newlines=True)
#            ).stdout.strip().split("\n")

# Print the output
print(gendf_files.stdout)

gendf_files/tendl_2017_Fe052.gendf
gendf_files/tendl_2017_Fe052m.gendf
gendf_files/tendl_2017_Fe053.gendf
gendf_files/tendl_2017_Fe053m.gendf
gendf_files/tendl_2017_Fe054.gendf
gendf_files/tendl_2017_Fe055.gendf
gendf_files/tendl_2017_Fe056.gendf
gendf_files/tendl_2017_Fe057.gendf
gendf_files/tendl_2017_Fe058.gendf
gendf_files/tendl_2017_Fe059.gendf
gendf_files/tendl_2017_Fe060.gendf
gendf_files/tendl_2017_Fe061.gendf
gendf_files/tendl_2017_Fe062.gendf
gendf_files/tendl_2017_Fe063.gendf
gendf_files/tendl_2017_Fe064.gendf
gendf_files/tendl_2017_Fe065m.gendf
gendf_files/tendl_2017_Fe56.gendf



In [22]:
import os
dir = './gendf_files'
os.path.isdir(dir)

True

In [28]:
subprocess.CalledProcessError(returncode= 2, cmd='ls')

subprocess.CalledProcessError()

In [6]:
path = 'gendf_files/tendl_2017_Fe052.gendf '
element = 'Fe'
element_index = path.rfind(element)
ext_index = path.find('.gendf')
A = path[element_index + len(element): ext_index]
A

'052'

In [1]:
import os

gendf_dir = './gendf_files'

if gendf_dir:
    print("success")

success


In [35]:
from logging_config import logger
import logging

logger.handlers.pop?

Signature: logger.handlers.pop(index=-1, /)
Docstring:
Remove and return item at index (default last).

Raises IndexError if list is empty or index is out of range.
Type:      builtin_function_or_method


In [38]:
79 - len('2024-07-15 15:21:17,929 - INFO All files:')

38

In [13]:
# Dictionary of elements in the Periodic Table
elements = [
    'H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
    'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca',
    'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr',
    'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
    'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
    'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
    'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th',
    'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm',
    'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds',
    'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og'
]
elements = dict(zip(elements, range(1, len(elements)+1)))
{v: k for k, v in elements.items()}
elements.keys()

dict_keys(['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og'])

In [10]:
from numpy import array

NP_dict   = {'n'     : array([-1      ,      0      ]), # neutron emission
             'p'     : array([ 0      ,     -1      ]), # proton emission
             'd'     : array([-1      ,     -1      ]), # deuteron emission
             't'     : array([-2      ,     -1      ]), # triton emission
             '3He'   : array([-1      ,     -2      ]), # helium-3 emission
             'α'     : array([-2      ,     -2      ]), # alpha emission
}

for particle in NP_dict.keys():
    print(particle)

n
p
d
t
3He
α


In [17]:
from pathlib import Path

save_directory = Path('./gendf_files')

if not save_directory.exists():
    save_directory.mkdir(parents=True)

In [22]:
str(Path.cwd()) + '/mt_table.csv'

'/filespace/e/esweinstein2/ALARA/src/DataLib/fendl32B_retrofit/mt_table.csv'

In [29]:
directory_dict = {
    'same' : '',
    'daug' : 'fendl32B_retrofit'
}

f'{directory_dict["daug"]}/mt_table.csv'

'fendl32B_retrofit/mt_table.csv'

In [32]:
Path.exists(Path('mt_table.csv'))

True

In [ ]:
import csv

def process_mt_table(csv_path):
    """
    Load in the mt_table.csv file which contains Table B.1 -
        "Reaction Type Numbers MT" from the ENDF-6 manual which can be found
        at https://www.oecd-nea.org/dbdata/data/manual-endf/endf102_MT.pdf.
        Given this, calculate the resultant change in KZA associated with each
        MT value in the table and tally the particle emissions associated with
        these reactions. Store all of this data in a dictionary of the format:
        {'MT' : {'Reaction' : (z , emission),
                 'delKZA' : change_in_KZA,
                 'Emitted Particles' : string_of_particles
                }
        }
    
    Arguments:
        csv_path (str): File path to mt_table.csv
            This should be in the same repository.
    
    Returns:
        mt_dict (dict): Dictionary formatted data structure for mt_table.csv,
            along with changes in KZA and emitted_particles.
    """
    csv_path = 'csv_path'
    mt_dict = {}

    with open(csv_path, 'r') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:
            mt_dict[row['MT']] = row['Reaction'].split(',')[1][:-1]

    return mt_dict

In [20]:
from string import Template

s = Template('$who likes $what')
s.substitute(who='tim', what = 'kung pao')
type(s)

string.Template

In [19]:
from groupr_tools import groupr_static_template, format_card
from string import Template

string = 'groupr\n'
for card_num, card in groupr_static_template().items():
    string += format_card(card_num, card, 11)
string += ' 0/\nstop'
print(string)
    

groupr
 20 21 0 31/
 17 0 11 0 1 1 1/
 /
 293.16
 0/
 /
 0/
 0/
stop


In [8]:
from string import Template
x = 'j $t'
s = Template(
    """
    text
    new line
    title
    """
)
title_index = s.template.find('title')
s.template[title_index : title_index + len('title')]

27

In [23]:
elements = [
    'H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
    'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca',
    'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr',
    'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
    'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
    'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
    'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th',
    'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm',
    'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds',
    'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og'
]
elements = dict(zip(elements, range(1, len(elements)+1)))

In [28]:
MFD = 3
from groupr_tools import establish_static_template

def fill_input_template(material_id, MTs, element, A, mt_dict, template):
    Z = str(elements[element]).zfill(2)
    title = f'"{Z}-{element}-{A} for TENDL 2017"'
    card9_lines = []

    for MT in MTs:
        mtname = mt_dict[str(MT)]['Reaction']
        card9_lines.append(f'{MFD} {MT} "{mtname}" /')
    card9 = '\n '.join(card9_lines).drop
    print(template.substitute(mat_id = material_id, title = title, reactions = card9))

    return template

import reaction_data as rxd
mt_dict = rxd.process_mt_data(rxd.load_mt_table('mt_table.csv'))
input_template = establish_static_template()
input_template = fill_input_template(125, [1,11,54], 'Fe', 56, mt_dict, input_template)

ImportError: cannot import name 'establish_static_template' from 'groupr_tools' (/filespace/e/esweinstein2/ALARA/src/DataLib/fendl32B_retrofit/groupr_tools.py)